## TODO

#### coding
- Adopt standard variable convention for op, op_str, etc.
    - matrixoperators are given name mat_op, singletrace are given st_op
    - op string names 'X', 'P' are op_str
    - operator tuples are op

- how to handle 'empty' operator SingleTraceOperator(data={():0})
- related, but maybe distinct: how to handle 'zero' operator like A - A = 0?

#### constraints
- inspect cyclic constraints, make sure they look good
- how to enforce reality/imaginarity of v_i (should I map P -> i P to make it Hermitian?)
- how should I impose that odd degree expectation values = 0 (and should I impose it directly?)
- how to build matrix version of cyclic constraints

#### optimization
- understand optimization used in Han et al
    - understand representation of \mathcal{M}_{ij} matrix as bootstrap tables
    - understand SDP relaxtion - is it worth just using that here?

In [1]:
from typing import Union, Self
from numbers import Number
from itertools import chain, product

import numpy as np
import sympy as sp

import scipy
from scipy.linalg import qr
from scipy.sparse import coo_matrix, csc_matrix
from scipy.sparse.linalg import splu, svds
from sksparse.cholmod import cholesky

from bmn.algebra import MatrixOperator, SingleTraceOperator, MatrixSystem
from bmn.linear_algebra import get_null_space, create_sparse_matrix_from_dict

## One Matrix Model

In [101]:
class BootstrapSystem():
    """
    _summary_
    """
    def __init__(
            self,
            matrix_system: MatrixSystem,
            hamiltonian: SingleTraceOperator,
            gauge: MatrixOperator,
            half_max_degree: int,
            tol: float = 1e-10
            ):
        self.matrix_system = matrix_system
        self.hamiltonian = hamiltonian
        self.gauge = gauge
        self.half_max_degree = half_max_degree
        self.operator_list = self.generate_operators(2*half_max_degree)
        self.operator_dict = {op: idx for idx, op in enumerate(self.operator_list)}
        if 2*self.half_max_degree < self.hamiltonian.max_degree:
            raise ValueError("2 * half_max_degree must be >= max degree of Hamiltonian.")
        self.param_dim = len(self.operator_dict)
        self.tol = tol

    def generate_operators(self, max_degree: int) -> list[str]:
        """
        Generate the list of operators used in the bootstrap, i.e.
            I, X, P, XX, XP, PX, PP, ...,
        up to and including strings of max_degree degree.

        Parameters
        ----------
        max_degree : int
            Maximum degree of operators to consider.

        Returns
        -------
        list[str]
            A list of the operators.
        """
        operators = {deg: [x for x in product(self.matrix_system.operator_basis, repeat=deg)] for deg in range(0, max_degree+1)}
        self.psd_matrix_dim = sum(len(value) for degree, value in operators.items() if degree <= self.half_max_degree)
        return [x for xs in operators.values() for x in xs] # flatten

    def single_trace_to_coefficient_vector(self, st_operator: SingleTraceOperator) -> np.ndarray:
        """
        TODO make sparse compatible
        Map a single trace operator to a vector of the coefficients.

        Parameters
        ----------
        st_operator : SingleTraceOperator
            The operator

        Returns
        -------
        np.ndarray
            The vector.
        """
        vec = [0]*self.param_dim
        for op, coeff in st_operator:
            idx = self.operator_dict[op]
            vec[idx] = coeff
        return np.asarray(vec)

    def generate_hamiltonian_constraints(self) -> list[SingleTraceOperator]:
        """
        Generate the Hamiltonian constraints <[H,O]>=0 for O single trace.

        Returns
        -------
        list[SingleTraceOperator]
            The list of constraint terms.
        """
        constraints = []
        for op in self.operator_list:
             constraints.append(self.matrix_system.single_trace_commutator(
                 st_operator1=self.hamiltonian,
                 st_operator2=SingleTraceOperator(data={op: 1})
                 ))
        return self.clean_constraints(constraints)

    def generate_gauge_constraints(self) -> list[SingleTraceOperator]:
        """
        Generate the Gauge constraints <tr(G O)>=0 for O a general matrix operator.
        Because G has max degree 2, this will create constraints involving terms
        with degree max_degree + 2. These will be discarded.

        Returns
        -------
        list[SingleTraceOperator]
            The list of constraint terms.
        """
        constraints = []
        for op in self.operator_list:
             constraints.append(
                 (self.gauge * MatrixOperator(data={op: 1})).trace()
                 )
        return self.clean_constraints(constraints)

    def generate_reality_constraints(self) -> list[SingleTraceOperator]:
        """
        Generate single trace constraints imposed by reality,
            <O^dagger> = <O>

        Returns
        -------
        list[SingleTraceOperator]
            The list of constraint terms.
        """
        constraints = []
        for op in self.operator_list:
             st_operator = SingleTraceOperator(data={op: 1})
             st_operator_dagger = self.matrix_system.hermitian_conjugate(operator=SingleTraceOperator(data={op: 1}))
             if len(st_operator - st_operator_dagger) > 0:
                 constraints.append(st_operator - st_operator_dagger)
        return self.clean_constraints(constraints)

    def generate_cyclic_constraints(self) -> list[SingleTraceOperator]:
        # TODO it might be nice to implement this using a DoubleTraceOperator class
        # I definitely don't like how it is impelemented now
        constraints = {}
        for idx, op in enumerate(self.operator_list):
            if len(op) > 1:
                # the LHS corresponds to single trace operators
                # note that in Eq S37, S38 their RHS contains single trace operators (k=1, k=r) in their notation
                eq_lhs = SingleTraceOperator(data={op: 1}) - SingleTraceOperator(data={tuple(op[1:]) + tuple(op[0]): 1})
                #eq_lhs += self.matrix_system.commutation_rules[(op_str[0], op_str[1])] * SingleTraceOperator(data={tuple(op_str[2:]): 1})
                #eq_lhs += self.matrix_system.commutation_rules[(op_str[0], op_str[len(op_str)-1])] * SingleTraceOperator(data={tuple(op_str[1:len(op_str)-1]): 1})

                # rhe RHS corresponds to double trace operators
                eq_rhs = []
                for k in range(1, len(op)):
                    commutator = self.matrix_system.commutation_rules[(op[0], op[k])]
                    #if idx == 7:
                    #    print(f'[{op[0]}, {op[k]}] = ', commutator)
                    eq_rhs.append([commutator, SingleTraceOperator(data={tuple(op[1:k]): 1}), SingleTraceOperator(data={tuple(op[k+1:]): 1})])
                constraints[idx] = {'lhs':eq_lhs, 'rhs':eq_rhs}
        return constraints

    def build_linear_constraints(self, return_matrix: bool=True) -> Union[list[SingleTraceOperator, coo_matrix]]:
        """
        Build the linear constraints. Each linear constraint corresponds to a
        linear combination of single trace operators that must vanish. The set
        of linear constraints may be numerically represented as a matrix L_{ij},
        where the first index runs over the set of all such constraints, and the
        second index runs over the set of single trace operators considered at this
        bootstrap, i.e., the constraint equations are

        L_{ij} v_j = 0.

        Parameters
        ----------
        return_matrix : bool, optional
            Flag used to control whether a numerical matrix is returned (default) or
            a list of SingleTraceOperators, which can be helpful for debugging.
            By default True

        Returns
        -------
        Union[list[SingleTraceOperator, coo_matrix]]
            The set of linear constraints.
        """

        empty_operator = SingleTraceOperator(data={():0})
        constraints = []

        # Hamiltonian constraints
        for st_operator in self.generate_hamiltonian_constraints():
            if st_operator != empty_operator:
                constraints.append({op: coeff for op, coeff in st_operator})

        # gauge constraints
        for st_operator in self.generate_gauge_constraints():
            if st_operator != empty_operator:
                constraints.append({op: coeff for op, coeff in st_operator})

        # reality constraints
        for st_operator in self.generate_reality_constraints():
            if st_operator != empty_operator:
                constraints.append({op: coeff for op, coeff in st_operator})

        # optionally return the constraints in a human-readable form
        if not return_matrix:
            return constraints

        # build the index-value dict
        index_value_dict = {}
        for idx_constraint, constraint_dict in enumerate(constraints):
            for op, coeff in constraint_dict.items():
                index_value_dict[(idx_constraint, self.operator_dict[op])] = coeff

        return create_sparse_matrix_from_dict(index_value_dict=index_value_dict, matrix_shape=(len(constraints), len(self.operator_list)))

    def build_quadratic_constraints(self):
        """
        TODO return type, pass in null space
        Build the quadratic constraints. The quadratic constraints are exclusively due to
        the cyclic constraints. The constraints can be written as

        A_{ijk} v_j v_k + B_{ij} v_j = 0.

        After imposing the linear constraints by transforming to the null basis, these become

        A'_{iab} u_a u_b + B'_{ia} u_a = 0,

        where A'_{iab} = M_{ijk} K_{ja} K_{kb}, B'_{ia} = B_{ij} K_{ja}

        Returns
        -------
        _type_
            _description_
        """

        linear_constraint_matrix = self.build_linear_constraints().todense()
        null_space_matrix = get_null_space(linear_constraint_matrix)

        empty_operator = SingleTraceOperator(data={():0})
        constraints = self.generate_cyclic_constraints()

        '''
         (7, ('X', 'X', 'X')): {'lhs': SingleTraceOperator(operators=[], coeff=[]),
        'rhs': [[0,
            SingleTraceOperator(operators=[()], coeff=[1]),
            SingleTraceOperator(operators=[('X',)], coeff=[1])],
        [0,
            SingleTraceOperator(operators=[('X',)], coeff=[1]),
            SingleTraceOperator(operators=[()], coeff=[1])]]},
        '''

        linear_terms = []
        quadratic_terms = []
        for constraint_idx, (operator_idx, term) in enumerate(constraints.items()):
            if term['lhs'] != empty_operator:
                linear_constraint_vector = self.single_trace_to_coefficient_vector(term['lhs'])

                quadratic_matrix = 1j * np.zeros((self.param_dim, self.param_dim)) # TODO make real eventually
                for i in range(len(term['rhs'])):
                    coeff = term['rhs'][i][0]
                    if np.abs(coeff) > self.tol:
                        quadratic_constraint_vector_1 = self.single_trace_to_vector(term['rhs'][i][1])
                        quadratic_constraint_vector_2 = self.single_trace_to_vector(term['rhs'][i][2])
                        quadratic_matrix += coeff * np.outer(quadratic_constraint_vector_2, quadratic_constraint_vector_2)

                # transform to null basis
                linear_constraint_vector = np.dot(linear_constraint_vector, null_space_matrix)
                quadratic_matrix = np.einsum('ia, ij, jb->ab', null_space_matrix, quadratic_matrix, null_space_matrix)
                #quadratic_constraint_vector_1 = np.dot(quadratic_constraint_vector_1, null_space_matrix)
                #quadratic_constraint_vector_2 = np.dot(quadratic_constraint_vector_2, null_space_matrix)

                # record
                linear_terms.append(linear_constraint_vector)
                quadratic_terms.append(quadratic_matrix)

        #for st_operator in self.generate_hamiltonian_constraints():
        #    if st_operator != empty_operator:
        #        constraints.append({op: coeff for op, coeff in st_operator})

        #if not return_matrix:
        #    return linear_terms
        return {'linear': linear_terms, 'quadratic': quadratic_terms}

        # build the index-value dict
        index_value_dict = {}
        for idx_constraint, constraint_dict in enumerate(linear_terms):
            for op, coeff in constraint_dict.items():
                index_value_dict[(idx_constraint, self.operator_dict[op])] = coeff

        linear_matrix = create_sparse_matrix_from_dict(index_value_dict=index_value_dict, matrix_shape=(len(constraints), len(self.operator_list)))

        return linear_matrix

    def clean_constraints(self, constraints: list[SingleTraceOperator]) -> list[SingleTraceOperator]:
        """
        Remove constraints that involve operators outside the basis set.

        Parameters
        ----------
        constraints : list[SingleTraceOperator]
            The single trace constraints.

        Returns
        -------
        list[SingleTraceOperator]
            The cleaned constraints.
        """
        cleaned_constraints = []
        for st_operator in constraints:
            if all([op in self.operator_list for op in st_operator.data]):
                cleaned_constraints.append(st_operator)
        return cleaned_constraints

    def build_bootstrap_table(self) -> None:
        """
        TODO consider passing in null space matrix as an argument.
        TODO figure out return type

        Creates the bootstrap table.

        The bootstrap matrix is

        M_{ij} = < tr(O^dagger_i O_j)>.

        Representing the set of single trace operators considered at this
        boostrap level as a vector v, this may be written as

        M_{ij} = v_{I(i,j)},

        where I(i,j) is an index map.

        For example, in the one-matrix case,
          i = 1 corresponds to the string 'X'
          i = 4 corresponds to the string 'XP'
          M_{14} = < tr(XXP) > = v_8

        After imposing the linear constraints, the variable v becomes

        v_i = K_{ij} u_j

        and the bootstrap matrix becomes

        M_{ij} = K_{I(i,j)k} u_k

        Define the bootstrap table as the 3-index array K_{I(i,j)k}.
        Note that this is independent of the single trace variables.

        ------------------------------------------------------------------------
        NOTE
        In https://doi.org/10.1103/PhysRevLett.125.041601, M is simplified by imposing discrete
        symmetries. Each operator is assigned an integer-valued degree, and only expectations
        with zero total degree are non-zero:

        degree_total( O_1^{dagger} O_2 ) = -degree(O_1) + degree(O_2).

        The degree function depends on the symmetry. In the two-matrix example the degree is
        the charge of the operators (in the A, B, C, D basis) under the SO(2) generators. In
        the one-matrix example, it is the charge under reflection symmetry (X, P) -> (-X, -P).

        Imposing the condition that the total degree is zero leads to M being block diagonal,
        provided that the operators are sorted by degree, as in

        O = [(degree -d_min operators), (degree -d + 1 operators), ..., (degree d_max operators)]

        The blocks will then corresponds to the different ways of pairing the two operators
        O_i^dagger O_j to form a degree zero operator.

        No discrete symmetries will be imposed here
        ------------------------------------------------------------------------
        Returns
        -------
        X
            X
        """
        linear_constraint_matrix = self.build_linear_constraints().todense()
        null_space_matrix = get_null_space(linear_constraint_matrix)
        bootstrap_dict = {}
        for idx1, op_str1 in enumerate(self.operator_list[:self.psd_matrix_dim]):
            for idx2, op_str2 in enumerate(self.operator_list[:self.psd_matrix_dim]):
                index_map = self.operator_dict[op_str1 + op_str2]
                for k in range(null_space_matrix.shape[1]):
                    bootstrap_dict[(idx1, idx2, k)] = null_space_matrix[index_map, k]
        return bootstrap_dict


In [102]:
matrix_system = MatrixSystem(
    operator_basis=['X', 'P'],
    commutation_rules_concise = {
        ('P', 'X'): -1j,
    }
)

# scale variables as P = sqrt(N) P', X = sqrt(N) X'
hamiltonian = SingleTraceOperator(
        data={("P", "P"): 1, ("X", "X"): 1, ("X", "X", "X", "X"): 7}
    )

# <tr G O > = 0 might need to be applied only for O with deg <= L-2
gauge = MatrixOperator(data={('X', 'P'): 1j, ('P', 'X'): -1j, ():1})

bootstrap = BootstrapSystem(
    matrix_system=matrix_system,
    hamiltonian=hamiltonian,
    gauge=gauge,
    half_max_degree=2
)
#bootstrap.operator_list[:bootstrap.psd_matrix_dim]
bootstrap.matrix_system.commutation_rules

{('X', 'X'): 0, ('P', 'X'): (-0-1j), ('X', 'P'): 1j, ('P', 'P'): 0}

In [103]:
bootstrap.param_dim

31

In [104]:
bootstrap.generate_cyclic_constraints()

{3: {'lhs': SingleTraceOperator(operators=[], coeff=[]),
  'rhs': [[0,
    SingleTraceOperator(operators=[()], coeff=[1]),
    SingleTraceOperator(operators=[()], coeff=[1])]]},
 4: {'lhs': SingleTraceOperator(operators=[('X', 'P'), ('P', 'X')], coeff=[1, -1]),
  'rhs': [[1j,
    SingleTraceOperator(operators=[()], coeff=[1]),
    SingleTraceOperator(operators=[()], coeff=[1])]]},
 5: {'lhs': SingleTraceOperator(operators=[('P', 'X'), ('X', 'P')], coeff=[1, -1]),
  'rhs': [[(-0-1j),
    SingleTraceOperator(operators=[()], coeff=[1]),
    SingleTraceOperator(operators=[()], coeff=[1])]]},
 6: {'lhs': SingleTraceOperator(operators=[], coeff=[]),
  'rhs': [[0,
    SingleTraceOperator(operators=[()], coeff=[1]),
    SingleTraceOperator(operators=[()], coeff=[1])]]},
 7: {'lhs': SingleTraceOperator(operators=[], coeff=[]),
  'rhs': [[0,
    SingleTraceOperator(operators=[()], coeff=[1]),
    SingleTraceOperator(operators=[('X',)], coeff=[1])],
   [0,
    SingleTraceOperator(operators=[('X',

In [105]:
len(bootstrap.generate_cyclic_constraints())

28

In [106]:
bootstrap.build_quadratic_constraints()['linear'][0].shape

(10,)

In [107]:
bootstrap.build_quadratic_constraints()['quadratic'][0].shape

(10, 10)

In [108]:
bootstrap.build_linear_constraints()

<36x31 sparse matrix of type '<class 'numpy.complex128'>'
	with 82 stored elements in COOrdinate format>

In [43]:
len(bootstrap.build_linear_constraints(return_matrix=False)), len(bootstrap.operator_dict)

(168, 127)

In [44]:
len(bootstrap.build_bootstrap_table())

7875

In [45]:
len([key for key, value in bootstrap.build_bootstrap_table().items() if np.abs(value) > 1e-10])

6715

In [48]:
OP1 = SingleTraceOperator(
    data={('X', 'X'): 1, ('P', 'X'): 1j * 7}
    )
print(OP1)

1 tr ('X', 'X') + 7j tr ('P', 'X')


In [49]:
bootstrap.single_trace_to_vector(OP1)

array([0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+7.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
      

In [ ]:
matrix_system.hermitian_conjugate(OP1)

In [ ]:
bootstrap.operator_dict

In [ ]:
len(bootstrap.build_linear_constraints(return_matrix=False)), len(bootstrap.operator_dict)

In [ ]:
bootstrap.operator_dict[('X', 'X', 'P', 'P')]

In [ ]:
empty_operator = SingleTraceOperator(data={():0})
bootstrap.generate_cyclic_constraints()

In [ ]:
bootstrap.build_quadratic_constraints(return_matrix=False)

In [ ]:
coo = bootstrap.build_linear_constraints(return_matrix=True)
mat = coo.todense()

In [ ]:
OP1 = SingleTraceOperator(
    data={('X1', 'X2'): 1, ('P1', 'X2'): 7}
    )
OP2 = SingleTraceOperator(
    data={('P1',): 1, ('P2',): 5 * 1j, ('X1', 'X1', 'P2',): -3 * 1j}
    )
print(OP1)
print(OP2)

In [ ]:
bootstrap.operator_list[0], bootstrap.generate_hamiltonian_constraints()[0]

In [ ]:
bootstrap.operator_list[1], bootstrap.generate_hamiltonian_constraints()[1]

In [ ]:
bootstrap.operator_list[2], bootstrap.generate_hamiltonian_constraints()[2]

In [ ]:
bootstrap.operator_list[3], 1j * 1/2 * bootstrap.generate_hamiltonian_constraints()[3]

In [ ]:
bootstrap.generate_hamiltonian_constraints()[3], bootstrap.generate_hamiltonian_constraints()[3].hermitian_conjugate()

In [ ]:
bootstrap.operator_list[0], -1j * bootstrap.generate_gauge_constraints()[0]

In [ ]:
# this reproduces Eq 14 in the main text (promote this to a unit test)
idx = bootstrap.operator_list.index(('X', 'P', 'P', 'P'),)
bootstrap.generate_cyclic_constraints()[idx]

## OLD

In [ ]:
nu = 1.0

matrix_system = MatrixSystem(
    operator_basis=['X0', 'X1', 'X2', 'P0', 'P1', 'P2'],
    commutation_rules_concise = {
        ('P0', 'X0'): -1j,
        ('P1', 'X1'): -1j,
        ('P2', 'X2'): -1j,
    }
)

# scale variables as P = sqrt(N) P', X = sqrt(N) X'
hamiltonian = SingleTraceOperator(
        data={
           # kinetic term
            ("P0", "P1"): 1,
            ("P1", "P1"): 1,
            ("P1", "P1"): 1,
            # quadratic term
            ('X0', 'X0'): nu**2 / 2,
            ('X1', 'X1'): nu**2 / 2,
            ('X2', 'X2'): nu**2 / 2,
            # cubic term
            ('X0', 'X1', 'X2'): 6 * 1j * nu,
            # quadratic term (XY)
            ('X0', 'X1', 'X0', 'X1'): - 1/4,
            ('X1', 'X0', 'X1', 'X0'): -1/4,
            ('X0', 'X1', 'X1', 'X0'): 1/4,
            ('X1', 'X0', 'X0', 'X1'): 1/4,
            # quadratic term (XZ) TODO check sign
            ('X0', 'X2', 'X0', 'X2'): - 1/4,
            ('X2', 'X0', 'X2', 'X0'): -1/4,
            ('X0', 'X2', 'X2', 'X0'): 1/4,
            ('X2', 'X0', 'X0', 'X2'): 1/4,
            # quadratic term (YZ)
            ('X1', 'X2', 'X1', 'X2'): - 1/4,
            ('X2', 'X1', 'X2', 'X1'): -1/4,
            ('X1', 'X2', 'X2', 'X1'): 1/4,
            ('X2', 'X1', 'X1', 'X2'): 1/4,
            }
            )

# <tr G O > = 0 might need to be applied only for O with deg <= L-2
gauge = MatrixOperator(
    data={
        ('X0', 'P0'): 1j,
        ('P0', 'X0'): -1j,
        ('X1', 'P1'): 1j,
        ('P1', 'X1'): -1j,
        ('X2', 'P2'): 1j,
        ('P2', 'X2'): -1j,
        ():1}
    )

bootstrap = BootstrapSystem(
    matrix_system=matrix_system,
    hamiltonian=hamiltonian,
    gauge=gauge,
    half_max_degree=2
)
#bootstrap.operator_list[:bootstrap.psd_matrix_dim]
bootstrap.matrix_system.commutation_rules

In [ ]:
bootstrap.build_linear_constraints()